In [1]:
import os
import operator
import gensim
import numpy as np
from gensim.models import HdpModel
from gensim.models.wrappers import LdaMallet
from pprint import pprint

C:\Users\Pcw\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
import spacy
from spacy.lang.ur import Urdu
parser = Urdu()

def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [3]:
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

In [4]:
import nltk
stopwords = set(nltk.corpus.stopwords.words('urdu'))

In [5]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in stopwords]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [6]:
import random
text_data = []
with open('news.csv', encoding = 'utf-8') as f:
    for line in f:
        tokens = prepare_text_for_lda(line)
        if random.random() > .99:
            print(tokens)
            text_data.append(tokens)

['ابوجا', 'دسمبر', 'نائیجیریا', 'دہشتگردوں', 'کردیا', 'ذرائع', 'ابلاغ', 'مطابق', 'نائیجیریا', 'مشرقی', 'بورنو', 'پسندوں', 'گھروں', 'نتیجہ', 'ہوگئے۔', 'شاہدین', 'مطابق', 'زیادہ', 'دوسری', 'پولیس', 'تصدیق', 'آوروں', 'پولیس', 'اسٹیشن', 'سرکاری', 'عمارت', 'گزشتہ', 'دوران', 'حملوں', 'زیادہ']
['انقرہ', 'نومبر', 'بحیرہ', 'مچھیروں', 'تارکین', 'اتوار', 'ذرائع', 'ابلاغ', 'بتایا', 'تارکین', 'تعداد', 'بحیرہ', 'شمالی', 'سمندری', 'تنزانیہ', 'مچھیروں', 'دیئے۔', 'دوسری', 'کنٹرول', 'جمنازیم', 'پہنچایا']
['نیوجرسی', 'ستمبر', 'ریکارڈ', 'مرتبہ', 'عالمی', 'چیمپئن', 'برازیل', 'ایکوا', 'انٹرنیشنل', 'فرینڈلی', 'دلچسپ', 'مقابلے', 'انٹرنیشنل', 'فرینڈلی', 'میچوں', 'برازیل', 'مسلسل', 'دوسری', 'کامیابی', 'برازیل', 'کولمبیا', 'کھیلے', 'برازیل', 'ایکوا', 'ٹیموں', 'درمیان', 'دلچسپ', 'مقابلہ', 'اعصاب', 'برازیلین', 'برازیل', 'ویلین', 'ایکوا', 'برتری', 'دلائی', 'برقرار', 'دوسرے', 'دونوں', 'ٹیموں', 'درمیان', 'کانٹے', 'مقابلہ', 'ایکوا', 'برتری', 'سرتوڑ', 'کامیاب', 'برازیل', 'برتری', 'بدولت', 'مقابلہ']
['بیروت', '21ستمبر',

In [7]:
from gensim import corpora
dictionary = corpora.Dictionary(text_data)

In [8]:
corpus = [dictionary.doc2bow(text) for text in text_data]

In [9]:
hdpmodel = HdpModel(corpus=corpus, id2word=dictionary)

In [10]:
hdpmodel.show_topics()

[(0,
  '0.030*نتیجہ + 0.028*سرکاری + 0.026*سوارہوکر + 0.023*کامیاب + 0.022*پنڈال + 0.022*پولیس + 0.022*ناکام + 0.020*جمنازیم + 0.019*پوائنٹ + 0.018*اسٹیشن + 0.017*چابکدستی + 0.016*بیروت + 0.016*دلائی + 0.016*عالمی + 0.016*ہوگئے۔ + 0.015*بنانے + 0.015*ابوجا + 0.014*آفتاب + 0.014*اجتماع + 0.013*انٹرنیشنل'),
 (1,
  '0.044*ٹیموں + 0.037*بورنو + 0.031*درمیان + 0.030*برازیلین + 0.028*سرکاری + 0.026*بدولت + 0.025*قریبی + 0.021*سرگرم + 0.020*ہوگئے۔ذرائع + 0.019*اتوار + 0.019*بمبارکوٹارگٹ + 0.019*پولیس + 0.018*انقرہ + 0.017*شمالی + 0.017*ہلاکتوں + 0.016*میچوں + 0.015*اسٹیشن + 0.014*نتیجہ + 0.013*تنزانیہ + 0.013*تعداد'),
 (2,
  '0.031*برازیلین + 0.030*مطابق + 0.028*سمندری + 0.027*لیڈرکاخطاب + 0.026*زیادہ + 0.026*واقعہ + 0.024*چیئرمین + 0.023*ذرائع + 0.022*خودکش + 0.021*ابوجا + 0.021*سرکاری + 0.020*پولیس + 0.020*عمرزئی + 0.019*آیاجس + 0.018*آوروں + 0.018*مقابلے + 0.017*عوامی + 0.017*شیرپاﺅ + 0.016*مچھیروں + 0.016*ابلاغ'),
 (3,
  '0.029*برازیل + 0.028*کولمبیا + 0.027*انٹرنیشنل + 0.024*اثناءحملہ + 

In [11]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

In [12]:
pyLDAvis.gensim.prepare(hdpmodel, corpus, dictionary)

C:\Users\Pcw\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster          Freq
topic                                                   
3      0.020339  0.009193       1        1  2.945629e+01
0      0.052173 -0.000110       2        1  2.566529e+01
2      0.031020  0.015349       3        1  1.771126e+01
8      0.011030  0.000544       4        1  1.192515e+01
13    -0.012019  0.024778       5        1  1.082261e+01
136   -0.060960  0.029675       6        1  3.453139e+00
1      0.009073  0.002022       7        1  6.180337e-01
4      0.025101  0.005927       8        1  1.502039e-01
5      0.007813  0.030956       9        1  8.972010e-02
6     -0.019218  0.018755      10        1  5.359179e-02
7     -0.055393 -0.000531      11        1  3.201155e-02
9     -0.037551  0.005759      12        1  1.063272e-02
10     0.050788  0.009307      13        1  5.912545e-03
11     0.009076 -0.024068      14        1  3.287781e-03
12     0.031770  0.002390      15        1  1.828234e-03
14    -0.011957  0.037384      16        1  5.145320e-04
15     0.035947 -0.026304      17        1  2.604138e-04
16     0.040012 -0.051695      18        1  1.318000e-04
17    -0.013501  0.013721      19        1  6.670633e-05
18     0.011613 -0.012566      20        1  3.376125e-05
19    -0.048659  0.018211      21        1  1.708717e-05
20    -0.034439  0.022023      22        1  8.648118e-06
21    -0.007489 -0.030631      23        1  4.376966e-06
22    -0.041308  0.018744      24        1  2.215260e-06
23    -0.048163 -0.023532      25        1  1.121182e-06
24     0.017702 -0.068663      26        1  5.674502e-07
25     0.015950 -0.001981      27        1  2.871966e-07
26     0.000110  0.015758      28        1  1.453553e-07
27    -0.027475 -0.025400      29        1  7.356688e-08
28     0.030625  0.040314      30        1  3.723350e-08
...         ...       ...     ...      ...           ...
119   -0.003803  0.021433     121        1  1.608608e-35
120    0.003130 -0.028091     122        1  8.043043e-36
121   -0.003125  0.027712     123        1  4.021521e-36
122    0.047698  0.025267     124        1  2.010761e-36
123    0.016432 -0.031295     125        1  1.005380e-36
124    0.001194  0.006853     126        1  5.026902e-37
125    0.019843 -0.014921     127        1  2.513451e-37
126    0.027059  0.042434     128        1  1.256725e-37
127   -0.012782  0.017113     129        1  6.283627e-38
128    0.000884 -0.022579     130        1  3.141814e-38
129    0.016731  0.028655     131        1  1.570907e-38
130   -0.019541 -0.003297     132        1  7.854534e-39
131    0.009264  0.027757     133        1  3.927267e-39
132   -0.004238 -0.005456     134        1  1.963634e-39
133   -0.026993 -0.000548     135        1  9.818168e-40
134   -0.004925  0.028665     136        1  4.909084e-40
135    0.001852 -0.013158     137        1  2.454542e-40
137   -0.030388  0.007956     138        1  6.136355e-41
138   -0.013787 -0.049307     139        1  3.068177e-41
139    0.031586  0.020074     140        1  1.534089e-41
140    0.024997  0.015971     141        1  7.670444e-42
141   -0.031109 -0.004179     142        1  3.835222e-42
142    0.030130  0.031642     143        1  1.917611e-42
143    0.004985 -0.038409     144        1  9.588055e-43
144   -0.025516 -0.059144     145        1  4.794027e-43
145   -0.012997 -0.001951     146        1  2.397014e-43
146   -0.039376 -0.032277     147        1  1.198507e-43
147    0.025412  0.050887     148        1  5.992534e-44
148    0.012205 -0.060611     149        1  2.996267e-44
149    0.007927 -0.009740     150        1  2.996267e-44

[150 rows x 5 columns], topic_info=      Category          Freq          Term     Total  loglift  logprob
term                                                                  
19     Default  2.000000e+00         نتیجہ  2.000000  30.0000  30.0000
46     Default  2.000000e+00      برازیلین  2.000000  29.0000  29.0000
42     Default  3.000000e+00     انٹرنیشنل  3.000000  28.0

In [15]:
def ret_top_model():
    top_topics = [(0, 0)]
    while top_topics[0][1] < 0.97:
        lm = HdpModel(corpus=corpus, id2word=dictionary)
        coherence_values = {}
        for n, topic in lm.show_topics(num_topics=-1, formatted=False):
            topic = [word for word, _ in topic]
            cm = gensim.models.CoherenceModel(topics=[topic], texts=corpus, dictionary=dictionary, window_size=10)
            coherence_values[n] = cm.get_coherence()
        top_topics = sorted(coherence_values.items(), key=operator.itemgetter(1), reverse=True)
    return lm, top_topics

In [16]:
lm, top_topics = ret_top_model()

C:\Users\Pcw\Anaconda3\lib\site-packages\gensim\topic_coherence\direct_confirmation_measure.py:193: RuntimeWarning: invalid value encountered in true_divide
  numerator = (co_occur_count / num_docs) + EPSILON
C:\Users\Pcw\Anaconda3\lib\site-packages\gensim\topic_coherence\direct_confirmation_measure.py:194: RuntimeWarning: invalid value encountered in true_divide
  denominator = (w_prime_count / num_docs) * (w_star_count / num_docs)
C:\Users\Pcw\Anaconda3\lib\site-packages\gensim\topic_coherence\direct_confirmation_measure.py:189: RuntimeWarning: invalid value encountered in true_divide
  co_doc_prob = co_occur_count / num_docs


In [17]:
pprint([lm.show_topic(topicid) for topicid, c_v in top_topics[:10]])

[[('لیڈرکاخطاب', 0.045683507412551165),
  ('21ستمبر', 0.03687020660827556),
  ('مقابلے', 0.03145608425436561),
  ('زیادہ', 0.027618262822152826),
  ('پولیس', 0.025820756826482022),
  ('مقابلہ', 0.025794532814965083),
  ('دوران', 0.02274179450250478),
  ('گزشتہ', 0.021987403559642824),
  ('حملوں', 0.02173150142270045),
  ('کانٹے', 0.021430689825899858),
  ('ریکارڈ', 0.01860000587132067),
  ('مطابق', 0.018170046032694273),
  ('جنگجو', 0.017711823778599516),
  ('نموادارہوا', 0.01695341986305698),
  ('بناکر', 0.01666367540366874),
  ('خودکش', 0.015915573374705956),
  ('ہوگئے۔ذرائع', 0.01563391833091704),
  ('اثناءحملہ', 0.01562998761937122),
  ('ابلاغ', 0.015129043916457974),
  ('پنڈال', 0.014493220288986535)],
 [('بیروت', 0.03424388646515368),
  ('متعدد', 0.029472334123126876),
  ('لبنان', 0.028340219428730027),
  ('نتیجے', 0.02753709072527901),
  ('مقابلہ', 0.02428785444129723),
  ('تنزانیہ', 0.024242532458643846),
  ('روانہ', 0.02312210967920612),
  ('کردیا۔پولیس', 0.021065194743865572)